In [ ]:
!pip install --cache-dir .cache torch-geometric torch-sparse torch-scatter --pre
import argparse
import sys

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

from logger import Logger


class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)


def train(model, x, y_true, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(x[train_idx])
    loss = F.nll_loss(out, y_true.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, x, y_true, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    # Create a mask for the specific nodes
    mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    mask[node_list] = True  # Set the specific nodes to True

    # Ensure split indices are tensors
    train_idx = torch.tensor(split_idx['train'], dtype=torch.long)
    valid_idx = torch.tensor(split_idx['valid'], dtype=torch.long)
    test_idx = torch.tensor(split_idx['test'], dtype=torch.long)

    # Use the mask to filter indices correctly for each split
    train_mask = mask[train_idx]
    valid_mask = mask[valid_idx]
    test_mask = mask[test_idx]

    # Filter the true and predicted labels
    y_true_train = data.y[train_idx][train_mask]
    y_pred_train = y_pred[train_idx][train_mask]

    y_true_valid = data.y[valid_idx][valid_mask]
    y_pred_valid = y_pred[valid_idx][valid_mask]

    y_true_test = data.y[test_idx][test_mask]
    y_pred_test = y_pred[test_idx][test_mask]

    # Debugging outputs
    print(f"Train: {y_true_train.numel()}, {y_pred_train.numel()}")
    print(f"Valid: {y_true_valid.numel()}, {y_pred_valid.numel()}")
    print(f"Test: {y_true_test.numel()}, {y_pred_test.numel()}")

    # Evaluate using the filtered true and predicted labels
    train_acc = evaluator.eval({'y_true': y_true_train, 'y_pred': y_pred_train})['acc'] if y_true_train.numel() > 0 else 0.0
    valid_acc = evaluator.eval({'y_true': y_true_valid, 'y_pred': y_pred_valid})['acc'] if y_true_valid.numel() > 0 else 0.0
    test_acc = evaluator.eval({'y_true': y_true_test, 'y_pred': y_pred_test})['acc'] if y_true_test.numel() > 0 else 0.0

    return train_acc, valid_acc, test_acc


def main():
    parser = argparse.ArgumentParser(description='OGBN-Arxiv (MLP)')
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--log_steps', type=int, default=1)
    parser.add_argument('--use_node_embedding', action='store_true')
    parser.add_argument('--num_layers', type=int, default=3)
    parser.add_argument('--hidden_channels', type=int, default=256)
    parser.add_argument('--dropout', type=float, default=0.5)
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--epochs', type=int, default=500)
    parser.add_argument('--runs', type=int, default=10)
    args = parser.parse_args()
    print(args)

    device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)

    node_list = [110223, 146929, 2940, 104544, 62326, 29759, 96890, 47025, 117732, 163206]

    dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor())

    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    data = data.to(device)
    split_idx = dataset.get_idx_split()
    # Filter the split indices based on your `node_list`
    train_idx = torch.tensor([n for n in split_idx['train'] if n in node_list], device=device)
    valid_idx = torch.tensor([n for n in split_idx['valid'] if n in node_list], device=device)
    test_idx = torch.tensor([n for n in split_idx['test'] if n in node_list], device=device)

    print(train_idx)
    print(valid_idx)
    print(test_idx)

    if args.use_node_embedding:
        embedding = torch.load('embedding.pt', map_location='cpu')
        x = torch.cat([x, embedding], dim=-1)
    x = x.to(device)

    y_true = data.y.to(device)
    train_idx = split_idx['train'].to(device)

    model = MLP(x.size(-1), args.hidden_channels, dataset.num_classes,
                args.num_layers, args.dropout).to(device)

    evaluator = Evaluator(name='ogbn-arxiv')
    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, x, y_true, train_idx, optimizer)
            result = test(model, data, {'train': train_idx, 'valid': valid_idx, 'test': test_idx}, evaluator, node_list)
            logger.add_result(run, result)


            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}%, '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()